## Imports

In [1]:
import pandas as pd
import numpy as np
from cobra.io import load_json_model
import re
from cobra.flux_analysis import single_gene_deletion
from cobra.flux_analysis import flux_variability_analysis
import functions
import math

## Transcriptomics Data

In [2]:
CCLE_expression = pd.read_csv('./data/CCLE_expression.csv', index_col=0)

In [3]:
CCLE_expression.head()

,TSPAN6 (7105),TNMD (64102),DPM1 (8813),SCYL3 (57147),C1orf112 (55732),FGR (2268),CFH (3075),FUCA2 (2519),GCLC (2729),NFYA (4800),...,H3C2 (8358),H3C3 (8352),AC098582.1 (8916),DUS4L-BCAP29 (115253422),C8orf44-SGK3 (100533105),ELOA3B (728929),NPBWR1 (2831),ELOA3D (100506888),ELOA3 (162699),CDR1 (1038)
ACH-001113,4.331992,0.000000,7.364397,2.792855,4.470537,0.028569,1.226509,3.042644,6.499686,4.739848,...,2.689299,0.189034,0.201634,2.130931,0.555816,0.0,0.275007,0.0,0.0,0.000000
ACH-001289,4.566815,0.584963,7.106537,2.543496,3.504620,0.000000,0.189034,3.813525,4.221104,3.481557,...,1.286881,1.049631,0.321928,1.464668,0.632268,0.0,0.014355,0.0,0.0,0.000000
ACH-001339,3.150560,0.000000,7.379032,2.333424,4.227279,0.056584,1.310340,6.687061,3.682573,3.273516,...,0.594549,1.097611,0.831877,2.946731,0.475085,0.0,0.084064,0.0,0.0,0.042644
ACH-001538,5.085340,0.000000,7.154109,2.545968,3.084064,0.000000,5.868143,6.165309,4.489928,3.956986,...,0.214125,0.632268,0.298658,1.641546,0.443607,0.0,0.028569,0.0,0.0,0.000000
ACH-000242,6.729145,0.000000,6.537607,2.456806,3.867896,0.799087,7.208381,5.569856,7.127014,4.568032,...,1.117695,2.358959,0.084064,1.910733,0.000000,0.0,0.464668,0.0,0.0,0.000000


In [4]:
CCLE_expression.columns = CCLE_expression.columns.map(functions.clean_gene_name)
CCLE_expression = CCLE_expression.loc[['ACH-000256', 'ACH-000713', 'ACH-000116', 'ACH-000520', 'ACH-000542', 'ACH-000091']]
CCLE_expression

,TSPAN6,TNMD,DPM1,SCYL3,C1ORF112,FGR,CFH,FUCA2,GCLC,NFYA,...,H3C2,H3C3,AC098582.1,DUS4L-BCAP29,C8ORF44-SGK3,ELOA3B,NPBWR1,ELOA3D,ELOA3,CDR1
ACH-000256,3.715893,0.000000,6.957450,1.985500,3.399171,0.137504,5.750607,4.849499,3.214125,3.308885,...,1.761285,1.130931,0.111031,2.104337,0.137504,0.000000,0.000000,0.137504,0.000000,0.0
ACH-000713,4.990501,0.000000,7.167017,1.761285,3.472488,0.084064,2.839960,6.684819,6.358080,5.019257,...,1.137504,3.160275,0.298658,2.056584,0.214125,0.000000,1.176323,0.000000,0.000000,0.0
ACH-000116,4.996841,0.000000,6.455656,2.298658,3.852998,0.028569,0.863938,6.507636,4.698774,3.802193,...,3.163499,1.959770,0.807355,1.744161,0.765535,0.000000,0.238787,0.028569,0.028569,0.0
ACH-000520,3.460743,0.000000,6.399000,1.855990,3.374344,0.028569,3.261531,6.094869,5.290203,3.947666,...,0.765535,0.400538,0.201634,0.695994,0.124328,0.000000,2.333424,0.000000,0.000000,0.0
ACH-000542,3.718088,0.000000,7.133810,1.922198,3.787641,0.000000,0.475085,6.115824,4.336997,2.946731,...,0.748461,0.941106,0.443607,1.794936,0.526069,0.000000,0.014355,0.000000,0.000000,0.0
ACH-000091,2.319040,0.070389,6.485910,1.974529,2.518535,0.201634,1.726831,5.352970,5.456806,3.803227,...,2.223423,0.070389,0.485427,1.263034,0.839960,0.028569,0.070389,0.028569,0.028569,0.0


In [5]:
model = load_json_model('./data/Recon3D.json')
model.solver = 'gurobi' 

Set parameter Username
Set parameter LicenseID to value 2732830
Academic license - for non-commercial use only - expires 2026-11-04


In [6]:
model_genes = {g.name.upper() for g in model.genes}
common_genes = [g for g in CCLE_expression.columns if g in model_genes]
CCLE_expression = CCLE_expression[common_genes]
CCLE_expression

,FUCA2,GCLC,CFTR,CYP51A1,LAP3,HS3ST1,AOC1,MTMR7,SLC7A2,AK2,...,THTPA,SULT1A3,NDUFA7,SLC6A14,KMT2B,CBSL,PIP4K2B,PIGW,SRD5A2,ACACA
ACH-000256,4.849499,3.214125,2.353323,5.859970,5.568640,0.000000,0.000000,1.117695,2.424922,7.401562,...,3.828835,3.042644,5.474436,0.028569,4.731726,1.189034,3.975447,3.061776,0.000000,4.689858
ACH-000713,6.684819,6.358080,0.014355,6.581653,5.732269,0.310340,0.687061,2.169925,2.182692,7.047451,...,3.142413,3.738768,5.184677,0.070389,3.691534,0.584963,3.244887,3.467279,0.070389,4.323370
ACH-000116,6.507636,4.698774,0.321928,5.955592,5.128046,0.378512,0.495695,1.137504,0.298658,8.079271,...,4.149747,4.121015,5.971773,0.070389,4.426936,1.395063,4.671859,3.786596,0.000000,6.464342
ACH-000520,6.094869,5.290203,0.014355,4.815575,5.593354,0.056584,0.028569,0.333424,3.415488,7.963359,...,3.372952,2.889474,5.756756,0.000000,4.204767,0.678072,4.408712,2.269033,0.028569,5.133810
ACH-000542,6.115824,4.336997,0.014355,4.828327,5.318317,0.097611,0.014355,0.344828,2.594549,7.572965,...,3.224966,1.731183,5.663345,0.000000,2.737687,3.965323,3.523562,2.867896,0.000000,4.067811
ACH-000091,5.352970,5.456806,0.150560,5.302319,5.909773,0.056584,0.042644,0.895303,3.463361,7.261249,...,2.266037,4.408712,6.204571,0.275007,4.455492,3.948601,4.280214,2.965323,0.014355,4.507795


In [7]:
def build_cell_line_model(model, cell_line, biomass_target, CCLE_expression):
    rxn_ids = []
    orig_lbs = []
    orig_ubs = []
    new_lbs = []
    new_ubs = []

    df = functions.rxn_essential(model)

    for rxn in model.reactions:

        rxn_id = rxn.id

        # store only reactions where something will change
        rxn_ids.append(rxn_id)
        lower_bound=model.reactions.get_by_id(rxn_id).lower_bound
        upper_bound=model.reactions.get_by_id(rxn_id).upper_bound
        orig_lbs.append(lower_bound)
        orig_ubs.append(upper_bound)

        
        if functions.is_defined_inmedia(rxn_id):
            lb, ub = lower_bound, upper_bound

        elif functions.is_essential_reaction(df, rxn_id):
            lb, ub = functions.open_bounds(rxn)

        elif functions.has_transcriptomics(rxn, CCLE_expression):
            rule_type = functions.classify_rule(rxn)

            bounds = functions.calculate_new_bounds(
                rxn, rule_type,
                cell_line,
                CCLE_expression
            )

            if bounds is None:
                # fallback: open bounds
                lb, ub = functions.open_bounds(rxn)
            else:
                lb, ub = bounds

        else:
            lb, ub = functions.open_bounds(rxn)
        

        if lb > ub:
            lb, ub = ub, lb
        new_lbs.append(lb)
        new_ubs.append(ub)
        
    df_bound = pd.DataFrame({
    "rxn_id": rxn_ids,
    "orig_lb": orig_lbs,
    "orig_ub": orig_ubs,
    "new_lb": new_lbs,
    "new_ub": new_ubs
    }).set_index("rxn_id")

    df_bound["changed"] = df_bound["orig_lb"] != df_bound["new_lb"]
    df_bound["changed"] |= df_bound["orig_ub"] != df_bound["new_ub"]
    df_changed = df_bound[df_bound["changed"] == True]
    
    for rxn_id, row in df_changed.iterrows():

        # Assign new bounds
        model.reactions.get_by_id(rxn_id).bounds = (row['new_lb'], row['new_ub'])

        sol = model.slim_optimize()

        # Feasibility check → restore old bounds
        if sol is None or math.isnan(sol) or sol < biomass_target:
            model.reactions.get_by_id(rxn_id).bounds = (row['orig_lb'], row['orig_ub'])
            print(f"[RESTORED] {rxn_id}")
        else:
            print(f"[OK] {rxn_id} → ({row['new_lb']},{row['new_ub']}) {sol}")
            

In [8]:
model = load_json_model('./data/Recon3D.json')
model.solver = 'gurobi' #you can use cplex or glpk as you whish, but gurobi and cplex are usually faster

In [9]:
df_fva_HG_min = pd.read_csv('./data/fva_exoflux_HG_min_sum.csv', index_col=0)

In [10]:
df_fva_HG_min[['minimum', 'maximum']] = df_fva_HG_min[['minimum', 'maximum']].fillna(0.0)
invalid_mask = df_fva_HG_min['minimum'] > df_fva_HG_min['maximum']
df_fva_HG_min.loc[invalid_mask, ['minimum', 'maximum']] = 0.0

In [11]:
sol = model.optimize()

In [12]:
biomass_target = sol.objective_value
biomass_target

755.003215550663

In [13]:
model.objective = 'BIOMASS_reaction'

In [14]:
for cell_name in ['ACH-000256','ACH-000713','ACH-000116']:

    print("***********************************************")
    print("Processing cell line:", cell_name)
    print("***********************************************")

    # reset model to HG low-grade bounds
    for index, row in df_fva_HG_min.iterrows():
        model.reactions.get_by_id(index).bounds = (row['minimum'], row['maximum'])

    build_cell_line_model(model, cell_name, biomass_target, CCLE_expression)
    
    # compute variability analysis
    deletion = single_gene_deletion(model)
    deletion.to_csv(f'./data/deletion_{cell_name}_HG.csv')

***********************************************
Processing cell line: ACH-000256
***********************************************


c:\Users\leabr\Ovarian-Cancer\functions.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_essential["rxn_ids"]=df_essential.ids.apply(lambda x: list(x)[0] if isinstance(x, (set, frozenset)) else x) # remove {} around ids


[RESTORED] 24_25DHVITD3tm
[RESTORED] 25HVITD3t
[RESTORED] 24_25VITD3Hm
[RESTORED] 24NPHte
[RESTORED] 10FTHF7GLUtl
[RESTORED] 11DOCRTSLtr
[RESTORED] 13DAMPPOX
[RESTORED] 24_25DHVITD2t
[RESTORED] 24_25DHVITD2tm
[RESTORED] 24_25DHVITD3t
[RESTORED] 25VITD2Hm
[RESTORED] 2AMACHYD
[RESTORED] 2AMACSULT
[RESTORED] 2AMADPTm
[RESTORED] 2MCITt
[RESTORED] 2OXOADOXm
[RESTORED] 2OXOADPTm
[RESTORED] 34DHPHEt
[RESTORED] 34DHXMANDACOX
[RESTORED] 34HPLFM
[RESTORED] 3AIBtm
[RESTORED] 3HAO
[RESTORED] 3MOXTYROX
[RESTORED] 3SALAOX
[RESTORED] 42A12BOOX
[RESTORED] 4HBZCOAFm
[RESTORED] 4HBZFm
[RESTORED] 4HDEBRISOQUINEte
[RESTORED] 4HOXPACDOX_NADP
[RESTORED] 4MPTNLte
[RESTORED] 4NPHSULT
[RESTORED] 4PYRDX
[RESTORED] 5ADTSTSTERONEGLCtr
[RESTORED] 5ADTSTSTERONESULT
[RESTORED] 5ADTSTSTERONEte
[RESTORED] 5FTHFt2_1
[RESTORED] 5HOXINOXDA
[RESTORED] 5THFtl
[RESTORED] 7DHCHSTEROLtr
[RESTORED] 7DHFtm
[RESTORED] A4GALTc
[RESTORED] A4GALTg
[RESTORED] A4GNT2g
[RESTORED] ABO1g
[RESTORED] ABO4g
[RESTORED] ABO5g
[RESTORED] ABO6

c:\Users\leabr\Ovarian-Cancer\functions.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_essential["rxn_ids"]=df_essential.ids.apply(lambda x: list(x)[0] if isinstance(x, (set, frozenset)) else x) # remove {} around ids


[RESTORED] 24_25DHVITD3tm
[RESTORED] 25HVITD3t
[RESTORED] 24_25VITD3Hm
[RESTORED] 24NPHte
[RESTORED] 10FTHF7GLUtl
[RESTORED] 11DOCRTSLtr
[RESTORED] 13DAMPPOX
[RESTORED] 24_25DHVITD2t
[RESTORED] 24_25DHVITD2tm
[RESTORED] 24_25DHVITD3t
[RESTORED] 25VITD2Hm
[RESTORED] 2AMACHYD
[RESTORED] 2AMACSULT
[RESTORED] 2AMADPTm
[RESTORED] 2MCITt
[RESTORED] 2OXOADOXm
[RESTORED] 2OXOADPTm
[RESTORED] 34DHPHEt
[RESTORED] 34DHXMANDACOX
[RESTORED] 34HPLFM
[RESTORED] 3AIBtm
[RESTORED] 3HAO
[RESTORED] 3MOXTYROX
[RESTORED] 3SALAOX
[RESTORED] 42A12BOOX
[RESTORED] 4HBZCOAFm
[RESTORED] 4HBZFm
[RESTORED] 4HDEBRISOQUINEte
[RESTORED] 4HOXPACDOX_NADP
[RESTORED] 4MPTNLte
[RESTORED] 4NPHSULT
[RESTORED] 4PYRDX
[RESTORED] 5ADTSTSTERONEGLCtr
[RESTORED] 5ADTSTSTERONESULT
[RESTORED] 5ADTSTSTERONEte
[RESTORED] 5FTHFt2_1
[RESTORED] 5HOXINOXDA
[RESTORED] 5THFtl
[RESTORED] 7DHCHSTEROLtr
[RESTORED] 7DHFtm
[RESTORED] A4GALTc
[RESTORED] A4GALTg
[RESTORED] A4GNT2g
[RESTORED] ABO1g
[RESTORED] ABO4g
[RESTORED] ABO5g
[RESTORED] ABO6

c:\Users\leabr\Ovarian-Cancer\functions.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_essential["rxn_ids"]=df_essential.ids.apply(lambda x: list(x)[0] if isinstance(x, (set, frozenset)) else x) # remove {} around ids


[RESTORED] 24_25DHVITD3tm
[RESTORED] 25HVITD3t
[RESTORED] 24_25VITD3Hm
[RESTORED] 24NPHte
[RESTORED] 10FTHF7GLUtl
[RESTORED] 11DOCRTSLtr
[RESTORED] 13DAMPPOX
[RESTORED] 24_25DHVITD2t
[RESTORED] 24_25DHVITD2tm
[RESTORED] 24_25DHVITD3t
[RESTORED] 25VITD2Hm
[RESTORED] 2AMACHYD
[RESTORED] 2AMACSULT
[RESTORED] 2AMADPTm
[RESTORED] 2MCITt
[RESTORED] 2OXOADOXm
[RESTORED] 2OXOADPTm
[RESTORED] 34DHPHEt
[RESTORED] 34DHXMANDACOX
[RESTORED] 34HPLFM
[RESTORED] 3AIBtm
[RESTORED] 3HAO
[RESTORED] 3MOXTYROX
[RESTORED] 3SALAOX
[RESTORED] 42A12BOOX
[RESTORED] 4HBZCOAFm
[RESTORED] 4HBZFm
[RESTORED] 4HDEBRISOQUINEte
[RESTORED] 4HOXPACDOX_NADP
[RESTORED] 4MPTNLte
[RESTORED] 4NPHSULT
[RESTORED] 4PYRDX
[RESTORED] 5ADTSTSTERONEGLCtr
[RESTORED] 5ADTSTSTERONESULT
[RESTORED] 5ADTSTSTERONEte
[RESTORED] 5FTHFt2_1
[RESTORED] 5HOXINOXDA
[RESTORED] 5THFtl
[RESTORED] 7DHCHSTEROLtr
[RESTORED] 7DHFtm
[RESTORED] A4GALTc
[RESTORED] A4GALTg
[RESTORED] A4GNT2g
[RESTORED] ABO1g
[RESTORED] ABO4g
[RESTORED] ABO5g
[RESTORED] ABO6

In [15]:
'''for cell_name in ['ACH-000256','ACH-000713','ACH-000116']:

    print("***********************************************")
    print("Processing cell line:", cell_name)
    print("***********************************************")

    # reset model to HG low-grade bounds
    for index, row in df_fva_HG_min.iterrows():
        model.reactions.get_by_id(index).bounds = (row['minimum'], row['maximum'])

    build_cell_line_model(model, cell_name, biomass_target, CCLE_expression)
    
    # compute variability analysis
    df_var = flux_variability_analysis(model, fraction_of_optimum=0.0)
    df_var.to_csv(f'./data/fva_pipeline_{cell_name}_HG.csv')'''

'for cell_name in [\'ACH-000256\',\'ACH-000713\',\'ACH-000116\']:\n\n    print("***********************************************")\n    print("Processing cell line:", cell_name)\n    print("***********************************************")\n\n    # reset model to HG low-grade bounds\n    for index, row in df_fva_HG_min.iterrows():\n        model.reactions.get_by_id(index).bounds = (row[\'minimum\'], row[\'maximum\'])\n\n    build_cell_line_model(model, cell_name, biomass_target, CCLE_expression)\n\n    # compute variability analysis\n    df_var = flux_variability_analysis(model, fraction_of_optimum=0.0)\n    df_var.to_csv(f\'./data/fva_pipeline_{cell_name}_HG.csv\')'

## Low Grade Models

In [16]:
model = load_json_model('./data/Recon3D.json')
model.solver = 'gurobi' #you can use cplex or glpk as you whish, but gurobi and cplex are usually faster

In [17]:
df_fva_LG_min = pd.read_csv('./data/fva_exoflux_LG_min_sum.csv', index_col=0)

In [18]:
df_fva_LG_min[['minimum', 'maximum']] = df_fva_LG_min[['minimum', 'maximum']].fillna(0.0)
invalid_mask = df_fva_LG_min['minimum'] > df_fva_LG_min['maximum']
df_fva_LG_min.loc[invalid_mask, ['minimum', 'maximum']] = 0.0

In [19]:
sol = model.optimize()

In [20]:
biomass_target = sol.objective_value
biomass_target

755.003215550663

In [21]:
model.objective = 'BIOMASS_reaction'

In [22]:
for cell_name in ['ACH-000520', 'ACH-000542', 'ACH-000091' ]: #59M, HEYA8, OV56
    print("***********************************************")
    print("Processing cell line:", cell_name)
    print("***********************************************")
    #reset model
    for index, row in df_fva_LG_min.iterrows():
        model.reactions.get_by_id(index).bounds = (row['minimum'], row['maximum'])
    
    build_cell_line_model(model, cell_name, biomass_target, CCLE_expression)
    #compute variability analysis
    deletion = single_gene_deletion(model)
    deletion.to_csv(f'./data/deletion_{cell_name}_LG.csv')

***********************************************
Processing cell line: ACH-000520
***********************************************


c:\Users\leabr\Ovarian-Cancer\functions.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_essential["rxn_ids"]=df_essential.ids.apply(lambda x: list(x)[0] if isinstance(x, (set, frozenset)) else x) # remove {} around ids


[RESTORED] 24_25DHVITD3tm
[RESTORED] 25HVITD3t
[RESTORED] 24_25VITD3Hm
[RESTORED] 24NPHte
[RESTORED] 10FTHF7GLUtl
[RESTORED] 11DOCRTSLtr
[RESTORED] 13DAMPPOX
[RESTORED] 24_25DHVITD2t
[RESTORED] 24_25DHVITD2tm
[RESTORED] 24_25DHVITD3t
[RESTORED] 25VITD2Hm
[RESTORED] 2AMACHYD
[RESTORED] 2AMACSULT
[RESTORED] 2AMADPTm
[RESTORED] 2MCITt
[RESTORED] 2OXOADOXm
[RESTORED] 2OXOADPTm
[RESTORED] 34DHPHEt
[RESTORED] 34DHXMANDACOX
[RESTORED] 34HPLFM
[RESTORED] 3AIBtm
[RESTORED] 3HAO
[RESTORED] 3MOXTYROX
[RESTORED] 3SALAOX
[RESTORED] 42A12BOOX
[RESTORED] 4HBZCOAFm
[RESTORED] 4HBZFm
[RESTORED] 4HDEBRISOQUINEte
[RESTORED] 4HOXPACDOX_NADP
[RESTORED] 4MPTNLte
[RESTORED] 4NPHSULT
[RESTORED] 4PYRDX
[RESTORED] 5ADTSTSTERONEGLCtr
[RESTORED] 5ADTSTSTERONESULT
[RESTORED] 5ADTSTSTERONEte
[RESTORED] 5FTHFt2_1
[RESTORED] 5HOXINOXDA
[RESTORED] 5THFtl
[RESTORED] 7DHCHSTEROLtr
[RESTORED] 7DHFtm
[RESTORED] A4GALTc
[RESTORED] A4GALTg
[RESTORED] A4GNT2g
[RESTORED] ABO1g
[RESTORED] ABO4g
[RESTORED] ABO5g
[RESTORED] ABO6

c:\Users\leabr\Ovarian-Cancer\functions.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_essential["rxn_ids"]=df_essential.ids.apply(lambda x: list(x)[0] if isinstance(x, (set, frozenset)) else x) # remove {} around ids


[RESTORED] 24_25DHVITD3tm
[RESTORED] 25HVITD3t
[RESTORED] 24_25VITD3Hm
[RESTORED] 24NPHte
[RESTORED] 10FTHF7GLUtl
[RESTORED] 11DOCRTSLtr
[RESTORED] 13DAMPPOX
[RESTORED] 24_25DHVITD2t
[RESTORED] 24_25DHVITD2tm
[RESTORED] 24_25DHVITD3t
[RESTORED] 25VITD2Hm
[RESTORED] 2AMACHYD
[RESTORED] 2AMACSULT
[RESTORED] 2AMADPTm
[RESTORED] 2MCITt
[RESTORED] 2OXOADOXm
[RESTORED] 2OXOADPTm
[RESTORED] 34DHPHEt
[RESTORED] 34DHXMANDACOX
[RESTORED] 34HPLFM
[RESTORED] 3AIBtm
[RESTORED] 3HAO
[RESTORED] 3MOXTYROX
[RESTORED] 3SALAOX
[RESTORED] 42A12BOOX
[RESTORED] 4HBZCOAFm
[RESTORED] 4HBZFm
[RESTORED] 4HDEBRISOQUINEte
[RESTORED] 4HOXPACDOX_NADP
[RESTORED] 4MPTNLte
[RESTORED] 4NPHSULT
[RESTORED] 4PYRDX
[RESTORED] 5ADTSTSTERONEGLCtr
[RESTORED] 5ADTSTSTERONESULT
[RESTORED] 5ADTSTSTERONEte
[RESTORED] 5FTHFt2_1
[RESTORED] 5HOXINOXDA
[RESTORED] 5THFtl
[RESTORED] 7DHCHSTEROLtr
[RESTORED] 7DHFtm
[RESTORED] A4GALTc
[RESTORED] A4GALTg
[RESTORED] A4GNT2g
[RESTORED] ABO1g
[RESTORED] ABO4g
[RESTORED] ABO5g
[RESTORED] ABO6

c:\Users\leabr\Ovarian-Cancer\functions.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_essential["rxn_ids"]=df_essential.ids.apply(lambda x: list(x)[0] if isinstance(x, (set, frozenset)) else x) # remove {} around ids


[RESTORED] 24_25DHVITD3tm
[RESTORED] 25HVITD3t
[RESTORED] 24_25VITD3Hm
[RESTORED] 24NPHte
[RESTORED] 10FTHF7GLUtl
[RESTORED] 11DOCRTSLtr
[RESTORED] 13DAMPPOX
[RESTORED] 24_25DHVITD2t
[RESTORED] 24_25DHVITD2tm
[RESTORED] 24_25DHVITD3t
[RESTORED] 25VITD2Hm
[RESTORED] 2AMACHYD
[RESTORED] 2AMACSULT
[RESTORED] 2AMADPTm
[RESTORED] 2MCITt
[RESTORED] 2OXOADOXm
[RESTORED] 2OXOADPTm
[RESTORED] 34DHPHEt
[RESTORED] 34DHXMANDACOX
[RESTORED] 34HPLFM
[RESTORED] 3AIBtm
[RESTORED] 3HAO
[RESTORED] 3MOXTYROX
[RESTORED] 3SALAOX
[RESTORED] 42A12BOOX
[RESTORED] 4HBZCOAFm
[RESTORED] 4HBZFm
[RESTORED] 4HDEBRISOQUINEte
[RESTORED] 4HOXPACDOX_NADP
[RESTORED] 4MPTNLte
[RESTORED] 4NPHSULT
[RESTORED] 4PYRDX
[RESTORED] 5ADTSTSTERONEGLCtr
[RESTORED] 5ADTSTSTERONESULT
[RESTORED] 5ADTSTSTERONEte
[RESTORED] 5FTHFt2_1
[RESTORED] 5HOXINOXDA
[RESTORED] 5THFtl
[RESTORED] 7DHCHSTEROLtr
[RESTORED] 7DHFtm
[RESTORED] A4GALTc
[RESTORED] A4GALTg
[RESTORED] A4GNT2g
[RESTORED] ABO1g
[RESTORED] ABO4g
[RESTORED] ABO5g
[RESTORED] ABO6

In [23]:
'''for cell_name in ['ACH-000520', 'ACH-000542', 'ACH-000091' ]: #59M, HEYA8, OV56
    print("***********************************************")
    print("Processing cell line:", cell_name)
    print("***********************************************")
    #reset model
    for index, row in df_fva_LG_min.iterrows():
        model.reactions.get_by_id(index).bounds = (row['minimum'], row['maximum'])
    
    build_cell_line_model(model, cell_name, biomass_target, CCLE_expression)
    #compute variability analysis
    df_var = flux_variability_analysis(model, fraction_of_optimum=0.0)
    df_var.to_csv(f'./data/fva_pipeline_{cell_name}_LG.csv')'''

'for cell_name in [\'ACH-000520\', \'ACH-000542\', \'ACH-000091\' ]: #59M, HEYA8, OV56\n    print("***********************************************")\n    print("Processing cell line:", cell_name)\n    print("***********************************************")\n    #reset model\n    for index, row in df_fva_LG_min.iterrows():\n        model.reactions.get_by_id(index).bounds = (row[\'minimum\'], row[\'maximum\'])\n\n    build_cell_line_model(model, cell_name, biomass_target, CCLE_expression)\n    #compute variability analysis\n    df_var = flux_variability_analysis(model, fraction_of_optimum=0.0)\n    df_var.to_csv(f\'./data/fva_pipeline_{cell_name}_LG.csv\')'